In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

In [ ]:
os.listdir("../input/movie-review-sentiment-analysis-kernels-only")

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
train = pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/train.tsv',sep='\t')
test = pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/test.tsv',sep='\t')
sub = pd.read_csv('../input/movie-review-sentiment-analysis-kernels-only/sampleSubmission.csv', sep=",")


In [ ]:
full_text = list(train['Phrase'].values)+list(test['Phrase'].values)
tk = Tokenizer(lower = True,filters = '')
tk.fit_on_texts(full_text)
train_tokenized = tk.texts_to_sequences(train['Phrase'])
test_tokenized = tk.texts_to_sequences(test['Phrase'])
y = train['Sentiment']

In [ ]:
max_len = 50
X_train = pad_sequences(train_tokenized,maxlen=max_len)
X_test = pad_sequences(test_tokenized,maxlen=max_len) # 最大长度为50 不足50的前方补零
embed_size = 300
batch_size = 1000

lr = 0.01
training_iter = X_train.shape[0]
n_inputs = 300
n_steps = 50
n_hidden_unit = 128
n_classes = len(set(y))

embedding_path = "../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec"
def get_coefs(word,*arr): return word,np.asarray(arr,dtype = 'float32')
embeding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))
word_index = tk.word_index


embedding_matrix = np.zeros((len(word_index)+1,embed_size))
for word,i in word_index.items():
    embedding_vector = embeding_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
from sklearn.preprocessing import OneHotEncoder # 将y 值转换为ont_hot 编码的形式
ohe = OneHotEncoder(sparse=False)
y_ohe = ohe.fit_transform(y.values.reshape(-1,1)) # 将其转换为列向量

def getTrainData(step ,batch_size):
    train_index = X_train[((step-1)*batch_size):(step * batch_size)]# shape = [batch_size,n_step]
    train_data = np.zeros(shape = [train_index.shape[0],train_index.shape[1],embed_size])

    for i in range(train_index.shape[0]):
        for j in range(train_index.shape[1]):
            for k in range(embed_size):
                train_data[i][j][k] = embedding_matrix[train_index[i][j]][k]

    train_label = y_ohe[((step-1)*batch_size):(step * batch_size)]

    # print(train_index.shape,train_label.shape)
    # print(train_index[:5],train_label[:5])
    return train_data,train_label

# getTrainData(1,batch_size)
def getTestData(step, batch_size):
    test_index = X_test[((step-1)*batch_size):(step * batch_size)]
    test_data = np.zeros(shape = [test_index.shape[0],test_index.shape[1],embed_size])
    for i in range(test_index.shape[0]):
        for j in range(test_index.shape[1]):
            for k in range(embed_size):
                test_data[i][j][k] = embedding_matrix[test_index[i][j]][k]
    test_y = y_ohe[0:test_index.shape[0]]
    return test_data,test_y
# tensorflow

x = tf.placeholder(tf.float32,shape = [None,n_steps,n_inputs])
y = tf.placeholder(tf.float32,shape = [None,n_classes])

weights = {
    "in":tf.Variable(tf.random_normal([n_inputs,n_hidden_unit])),
    "out":tf.Variable(tf.random_normal([n_hidden_unit,n_classes]))
}
biases ={
    "in": tf.Variable(tf.constant(0.1,shape =[n_hidden_unit,])),
    "out":tf.Variable(tf.constant(0.1,shape = [n_classes,]))
}

def RNN(X, weights, biases):
    # X.shape = n_batch,n_steps,n_inputs
    X = tf.reshape(X, [-1,n_inputs])
    # X.shape = [batch_size *28, 28]
    X_in = tf.matmul(X, weights["in"])+ biases["in"]
    # X,shape = [batch_size,n_steps,n_hidden_unit]
    X_in = tf.reshape(X_in , shape = [-1, n_steps, n_hidden_unit])  #(n_batch, 28,128 hidden_size)

    # cell
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden_unit,forget_bias = 1.0, state_is_tuple = True)

    #_init_state =lstm_cell.zero_state(batch_size, dtype = tf.float32)

    outputs, states = tf.nn.dynamic_rnn(lstm_cell , X_in, dtype = tf.float32, time_major =False)

    # result = tf.matmul(states[-1], weights['out']) + biases['out']
    outputs = tf.unstack(tf.transpose(outputs, [1, 0, 2]))

    results = tf.matmul(outputs[-1], weights['out']) + biases['out']
    return results;

pred = RNN(x, weights, biases)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred,labels = y))
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

correct_pred = tf.argmax(pred, 1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2):
        if i == 1: # reverse data 
            reversed_train = reversed(X_train)
            reversed_y = reversed(y_ohe)
            X_train = np.array(list(reversed_train))
            y_ohe = np.array(list(reversed_y))
        step = 1
        while step * batch_size < training_iter:
            train_x,train_y = getTrainData(step ,batch_size)
            sess.run([train_op],feed_dict = {
                x:train_x,
                y:train_y
            })
            if step % 100 ==0:
                print(sess.run([cost],feed_dict = {
                    x: train_x,
                    y: train_y
                }))
            step+=1
    ans = []
    step = 1
    while step * batch_size < X_test.shape[0]:
        test_data,test_y = getTestData(step, batch_size)
        tmp = sess.run([correct_pred],feed_dict ={
         x:test_data,
         y: test_y
        })
        ans.extend(tmp[0])
        step +=1
    test_data, test_y = getTestData(step, batch_size)
    tmp = sess.run([correct_pred], feed_dict = {
        x: test_data,
        y: test_y
    })
    ans.extend(tmp[0])
    sub['Sentiment'] = ans
    sub.to_csv("blend1.csv", index = False)
